In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os
import requests
import time
import pandas as pd

In [2]:
# ChromeDriver 옵션 설정
def initialize_webdriver():
    options = Options()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 제어 메시지 숨기기
    return webdriver.Chrome(options=options)

#### 가게 상세 페이지 리뷰 크롤링함수

In [11]:
def extract_visitor_reviews(driver, max_reviews=100):
    reviews = []  # 방문자 리뷰 데이터를 저장할 리스트
    review_count = 0  # 현재까지 수집된 리뷰의 개수
    visited_reviews = set()  # 이미 수집한 리뷰를 추적하기 위한 set (중복 방지)

    try:
        while review_count < max_reviews:
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
            
            # 첫 번째 페이지에서 리뷰 크롤링
            review_elements = soup.select("ul.list_evaluation > li")
            
            # 새로운 리뷰만 수집 (이미 수집된 리뷰는 제외)
            for review_element in review_elements:
                if review_count >= max_reviews:
                    break  # 최대 리뷰 개수에 도달하면 종료
                
                # 리뷰 텍스트 추출
                comment = review_element.select_one("p.txt_comment")
                comment_text = comment.get_text(strip=True) if comment else "리뷰 없음"
                
                if comment_text not in visited_reviews:
                    reviews.append(comment_text)
                    visited_reviews.add(comment_text)  # 새로 수집한 리뷰를 기록
                    review_count += 1
            
            # 만약 리뷰 개수가 충분치 않다면 "후기 더보기" 버튼을 클릭하여 더 많은 리뷰 로드
            if review_count < max_reviews:
                try:
                    # "후기 더보기" 버튼이 있을 경우 클릭
                    load_more_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.link_more"))
                    )
                    load_more_button.click()
                    time.sleep(3)  # 페이지 로딩 대기
                    print(f"{review_count}개의 리뷰를 수집했습니다. 더 많은 리뷰를 로딩 중...")
                except Exception as e:
                    print(f"후기 더보기 버튼 클릭 중 오류 발생: {e}")
                    break  # "더보기" 버튼이 없거나 클릭이 안 되는 경우 종료

        return reviews

    except Exception as e:
        print(f"방문자 리뷰 수집 중 오류 발생: {e}")
        return ["후기가 제공되지 않는 가게입니다."]

In [12]:
# 메인 실행
if __name__ == "__main__":
    driver = initialize_webdriver()
    try:
        # 1. 카카오맵 접속
        print("카카오맵 접속 중...")
        driver.get("https://map.kakao.com/")
        time.sleep(2)
        print("카카오맵 접속 완료!")

        # 2. 검색어 입력
        place_name = "쿳사 연희"  # 검색할 가게명
        print("검색창에 가게명 입력 중...")
        search_box = driver.find_element(By.ID, "search.keyword.query")
        search_box.send_keys(place_name)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)
        print(f"'{place_name}' 검색 완료!")

        # 3. 투명 레이어 제거
        try:
            dimmed_layer = driver.find_element(By.ID, "dimmedLayer")
            driver.execute_script("arguments[0].style.display = 'none';", dimmed_layer)
            print("투명 레이어 제거 완료!")
        except Exception as e:
            print("투명 레이어가 존재하지 않습니다. 계속 진행합니다.")

        # 4. 검색 결과에서 상세 페이지 링크 클릭
        print("검색 결과에서 상세 페이지 링크 클릭 대기 중...")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.MediumTooltip a"))
        )
        print("상세 페이지 링크 찾기 완료!")

        print("상세 페이지 링크 클릭 중...")
        detail_link = driver.find_element(By.CSS_SELECTOR, "div.MediumTooltip a")
        detail_url = detail_link.get_attribute("href")  # 링크 주소 추출
        driver.get(detail_url)  # 상세페이지로 이동
        time.sleep(2)
        print("상세 페이지로 이동 완료!")

        # 리뷰 크롤링
        reviews = extract_visitor_reviews(driver, max_reviews=100)
        if reviews:
            print(reviews)
        else:
            print("리뷰 크롤링 결과 없음.")

    except Exception as e:
        print(f"오류 발생: {e}")
    finally:
        driver.quit()
        print("브라우저 종료 완료.")

카카오맵 접속 중...
카카오맵 접속 완료!
검색창에 가게명 입력 중...
'쿳사 연희' 검색 완료!
투명 레이어 제거 완료!
검색 결과에서 상세 페이지 링크 클릭 대기 중...
상세 페이지 링크 찾기 완료!
상세 페이지 링크 클릭 중...
상세 페이지로 이동 완료!
3개의 리뷰를 수집했습니다. 더 많은 리뷰를 로딩 중...
3개의 리뷰를 수집했습니다. 더 많은 리뷰를 로딩 중...
3개의 리뷰를 수집했습니다. 더 많은 리뷰를 로딩 중...
3개의 리뷰를 수집했습니다. 더 많은 리뷰를 로딩 중...
3개의 리뷰를 수집했습니다. 더 많은 리뷰를 로딩 중...
3개의 리뷰를 수집했습니다. 더 많은 리뷰를 로딩 중...
3개의 리뷰를 수집했습니다. 더 많은 리뷰를 로딩 중...
브라우저 종료 완료.


KeyboardInterrupt: 